In [1]:
import pandas as pd
import itertools
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
buildings_all = pd.read_csv("original_data/PublicLEEDProjectDirectory.csv", skiprows=3, encoding="ISO-8859-1")
buildings_all = buildings_all[buildings_all['Isconfidential'] == 'No']
buildings_2009 = buildings_all[buildings_all["LEEDSystemVersionDisplayName"] == "LEED-NC v2009"] 

/Users/trippshealy/Gdrive/research/leed_countries/.direnv/python-3.4.3/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
def my_filter_2009(df):
    countries = df.dropna(subset = ['PointsAchieved'])
    countries['PointsAchieved'] = countries['PointsAchieved'].convert_objects(convert_numeric=True)
    countries = countries[countries['PointsAchieved'] > 39]
    return(countries)

In [4]:
filtered = my_filter_2009(buildings_2009)

/Users/trippshealy/Gdrive/research/leed_countries/.direnv/python-3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
turkey = filtered[filtered['Country'] == 'TR']
usa = filtered[filtered['Country'] == 'US']
china = filtered[filtered['Country'] == 'CN']
brazil = filtered[filtered['Country'] == 'BR']

In [7]:
turkey.groupby('ProjectTypes').count()['ID']

ProjectTypes
Core Learning Space: College/University              6
Core Learning Space: Other classroom education       1
Industrial Manufacturing                             9
Laboratory                                           2
Lodging: Dormitory                                   1
Lodging: Hotel/Motel/Resort, Full Service            5
Lodging: Hotel/Motel/Resort, Select Service          2
Lodging: Inn                                         1
Multi-Family Residential: Apartment                  3
Multi-Family Residential: Condominium                5
Office: Administrative/Professional                 10
Office: Government                                   1
Office: Mixed-Use                                    2
Office: Other Office                                 2
Public Assembly: Other Assembly                      3
Public Assembly: Recreation                          1
Public Assembly: Social/Meeting                      1
Service: Other Service                              

In [9]:
china.groupby('ProjectTypes').count()['ID']

ProjectTypes
Core Learning Space: College/University                          1
Core Learning Space: Other classroom education                   1
Datacenter                                                       1
Health Care: Clinic/Other Outpatient                             1
Healthcare: Nursing Home/ Assisted Living                        1
Industrial Manufacturing                                        43
Laboratory                                                       6
Lodging: Hotel/Motel/Resort, Full Service                        5
Lodging: Hotel/Motel/Resort, Select Service                      1
Multi-Family Residential: Apartment                              2
Multi-Family Residential: Condominium                            1
Office: Administrative/Professional                             23
Office: Financial                                                1
Office: Government                                               2
Office: Mixed-Use                                

In [10]:
brazil.groupby('ProjectTypes').count()['ID']

ProjectTypes
Data Center                                      3
Datacenter                                       1
Health Care: Outpatient, Office (Diagnostic)     1
Healthcare: Inpatient                            1
Industrial Manufacturing                         4
Office: Administrative/Professional             12
Office: Financial                                1
Office: Government                               2
Office: Mixed-Use                                1
Office: Other Office                             6
Public Assembly: Entertainment                   3
Public Assembly: Library                         1
Public Assembly: Social/Meeting                  1
Public Assembly: Stadium/Arena                   8
Retail: Bank Branch                              1
Retail: Other Retail                             1
Service: Other Service                           2
Service: Vehicle Service/Repair                  1
Warehouse: Self Storage Units                    1
Name: ID, dtype: i

In [8]:
usa.groupby('ProjectTypes').count()['ID']

ProjectTypes
Animal Care                                                       1
Assembly                                                          2
Campus (corp/school)                                              2
Commercial Office                                                30
Community Dev.                                                    3
Core Learning Space: College/University                         378
Core Learning Space: K-12 Elementary/Middle School                1
Core Learning Space: K-12, Elementary/Middle School               1
Core Learning Space: K-12, High School                            3
Core Learning Space: Other classroom education                   28
Core Learning Space: Preschool/Daycare                           39
Data Center                                                       5
Datacenter                                                       22
Daycare                                                           1
Financial & Comm.                  

## Owner Type

In [11]:
turkey.groupby('OwnerTypes').count()['ID']

OwnerTypes
Community Development Corporation or Non     1
Corporate: Privately Held                   30
Corporate: Publicly Traded                   3
Educational: College, Private                2
Educational: College, Public                 1
Educational: University, Private             3
Government Use: Federal                      2
Government Use: Local, City                  1
Government Use: Local, County                2
Government Use: State                        2
Investor: Individual/Family                  1
Investor: REIT, Publicly traded              4
Non-Profit (that do not fit into other c     1
Name: ID, dtype: int64

In [12]:
china.groupby('OwnerTypes').count()['ID']

OwnerTypes
Community Development Corporation o          1
Corporate: Privately Held                   43
Corporate: Publicly Traded                  67
Educational: College, Public                 1
Government Use: Local, City                  6
Government Use: Other (utility, air          1
Government Use: Other (utility, airport,     2
Government Use: State                        2
Investor: Investment Manager                 3
Investor: REIT, Publicly traded              2
Non-Profit (that do not fit into other c     1
Name: ID, dtype: int64

In [13]:
brazil.groupby('OwnerTypes').count()['ID']

OwnerTypes
Community Development Corporation or Non     1
Corporate: Privately Held                   25
Corporate: Publicly Traded                   1
Government Use: Local, City                  1
Government Use: Other (utility, air          1
Government Use: State                        3
Investor: Bank                               1
Non-Profit (that do not fit into other c     3
Name: ID, dtype: int64

In [14]:
usa.groupby('OwnerTypes').count()['ID']

OwnerTypes
Business Improvement District            2
Community Development Corporation o     24
Corporate: Privately Held              592
Corporate: Publicly Traded             235
Educational: College, Private          103
Educational: College, Public           146
Educational: Community College, Pri      3
Educational: Community College, Pub     68
Educational: Early Childhood Educat      7
Educational: K-12 School, Private       14
Educational: K-12 School, Public        11
Educational: University, Private       109
Educational: University, Public        216
Government Use: Federal                354
Government Use: Local, City            255
Government Use: Local, County          129
Government Use: Local, Public Housi      5
Government Use: Other (utility, air     49
Government Use: State                  158
Investor: Bank                          54
Investor: Endowment                      2
Investor: Equity Fund                    5
Investor: Individual/Family             46


In [20]:
def filter_owner(df):
    return(df[df["OwnerTypes"] == 'Corporate: Privately Held'])
    

In [21]:
turkey_private = filter_owner(turkey)
usa_private = filter_owner(usa)
china_private = filter_owner(china)
brazil_private = filter_owner(brazil)

## Normal Distribution Test 

In [ ]:
import scipy.stats as stats

In [39]:
stats.normaltest(turkey_private.PointsAchieved)

NormaltestResult(statistic=4.0493412580840111, pvalue=0.13203732528688755)

In [40]:
stats.normaltest(usa_private.PointsAchieved)

NormaltestResult(statistic=106.81800886677151, pvalue=6.3791687472682691e-24)

### USA data is not normally distributed 

In [41]:
stats.normaltest(china_private.PointsAchieved)

NormaltestResult(statistic=1.2756782547669323, pvalue=0.52843306775210641)

In [42]:
stats.normaltest(brazil_private.PointsAchieved)

NormaltestResult(statistic=2.428493115546241, pvalue=0.29693365238656422)

## Statisical Difference in Privately held Companies by Country

In [24]:
a_list = ['TR', 'US', 'CN', 'BR']

In [46]:
my_list = itertools.combinations(a_list, 2)
list(my_list)

[('TR', 'US'),
 ('TR', 'CN'),
 ('TR', 'BR'),
 ('US', 'CN'),
 ('US', 'BR'),
 ('CN', 'BR')]

In [34]:
def my_t_test(df1, df2, c1, c2):
        one = df1[df1['Country'] == c1].PointsAchieved
        two = df2[df2['Country'] == c2].PointsAchieved
        t, p = ttest_ind(one, two, equal_var=False)
        print("{}, {}\n t {}, p {}".format(c1, c2, t, p))
#         if p <= 0.05:
#             stat_difference.append({i: (t, p)})
#     return(stat_difference)

In [35]:
my_t_test(turkey_private, usa_private, 'TR', 'US')
my_t_test(turkey_private, china_private, 'TR', 'CN')
my_t_test(turkey_private, brazil_private, 'TR', 'BR')
my_t_test(usa_private, china_private, 'US', 'CN')
my_t_test(usa_private, brazil_private, 'US', 'BR')
my_t_test(china_private, brazil_private, 'CN', 'BR')

TR, US
 t 4.643908223483067, p 5.638557687964413e-05
TR, CN
 t 0.24100620169474948, p 0.8104081077921602
TR, BR
 t 3.0467075834925725, p 0.003780096119642822
US, CN
 t -5.782041872308198, p 4.873129705171188e-07
US, BR
 t -1.7693664842549446, p 0.08691070553139825
CN, BR
 t 3.405440858065395, p 0.0011344508354554346


Drastically different in how they perform. 

### Can't look at USA p-values

## Mann Whitney U Test

In [48]:
my_list = itertools.combinations(a_list, 2)
list(my_list)

[('TR', 'US'),
 ('TR', 'CN'),
 ('TR', 'BR'),
 ('US', 'CN'),
 ('US', 'BR'),
 ('CN', 'BR')]

In [52]:
print("{}, {}\n {}".format('TR', 'US', stats.mannwhitneyu(turkey_private.PointsAchieved, usa_private.PointsAchieved)))
print("{}, {}\n {}".format('TR', 'CN', stats.mannwhitneyu(turkey_private.PointsAchieved, china_private.PointsAchieved)))
print("{}, {}\n {}".format('TR', 'BR', stats.mannwhitneyu(turkey_private.PointsAchieved, brazil_private.PointsAchieved)))
print("{}, {}\n {}".format('US', 'CN', stats.mannwhitneyu(usa_private.PointsAchieved, china_private.PointsAchieved)))
print("{}, {}\n {}".format('US', 'BR', stats.mannwhitneyu(usa_private.PointsAchieved, brazil_private.PointsAchieved)))
print("{}, {}\n {}".format('CN', 'BR', stats.mannwhitneyu(china_private.PointsAchieved, brazil_private.PointsAchieved)))

TR, US
 MannwhitneyuResult(statistic=4323.5, pvalue=1.0190524052380321e-06)
TR, CN
 MannwhitneyuResult(statistic=631.5, pvalue=0.4419573210567016)
TR, BR
 MannwhitneyuResult(statistic=208.5, pvalue=0.0024271230944570591)
US, CN
 MannwhitneyuResult(statistic=6168.5, pvalue=7.9101733986702824e-09)
US, BR
 MannwhitneyuResult(statistic=5853.5, pvalue=0.03812763362142884)
CN, BR
 MannwhitneyuResult(statistic=291.0, pvalue=0.00085886848551532677)
